#What we cover in this notebook:
* Introduction to tensors
* Getting information from Tensors
* Manipulating Tensors
* Tensors and NumPy
* Using @tf.function (to speed up regular python functions)
* Using GPUs with TensorFlow
*Exercises


# Import TensorFlow

In [1]:
import tensorflow as tf
print(tf.__version__)

2.15.0


# Creating tensors with `tf.constant()`


In [2]:
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Check the numbe of dimensions of a tensor
scalar.ndim

0

In [4]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# Check the dimension of our vector
vector.ndim

1

In [6]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7],
                     [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [7]:
matrix.ndim

2

In [8]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype = tf.float16) #specify the data type with dtype

another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
# The number of dimensions of another_matrix
another_matrix.ndim

2

In [10]:
# Creating a Tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [11]:
tensor.ndim

3

# Creating tensors with `tf.Variable`


In [12]:
# Ceate the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [13]:
# Lets try change one of the elements in our changeable tensor
try:
  unchangeable_tensor[1] = 7
  unchangeable_tensor
except Exception as e:
  print(e)

'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment


In [14]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

# Creating Random tensors

In [15]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) #seed for reproducability
random_1 = random_1.normal(shape = (3, 2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [16]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape = (3, 2))
random_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [17]:
random_1 == random_2

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>

# Shuffle the order of elements in a tensor

In [18]:
# Shuffle a tensor (valuable for when you want to shuffle your data so that the inherent)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])

# Shuffle our non_shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  5],
       [ 3,  4]], dtype=int32)>

In [19]:
# Setting both global and operation seed for a random sequence
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed = 42) # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

# Other ways to make tensors

In [20]:
# Creating a tensor with all zeros
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [21]:
# Creating a tensor with all zeros
tf.zeros(shape = (3, 4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

# Numpy arrays into tensors
The main difference betwween numpy arrays and tensorflow tensors is that tensors can be run in gpu must faster.

In [22]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype = np.int32) # create a NumPy array within the range of 1 and 24
numpy_A

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non_capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [23]:
A = tf.constant(numpy_A)
A

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [24]:
A = tf.constant(numpy_A, shape = (2, 3, 4))
B = tf.constant(numpy_A, shape = (2, 2, 6))

A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2, 6), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4,  5,  6],
         [ 7,  8,  9, 10, 11, 12]],
 
        [[13, 14, 15, 16, 17, 18],
         [19, 20, 21, 22, 23, 24]]], dtype=int32)>)

# Getting information from tensors

* Shape
* Rank
* Axis or Dimension
* Size

In [25]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape = [2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [26]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [27]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [28]:
# Get various output of our tensors
print('Datatype of every element:', rank_4_tensor.dtype)
print('Number of dimensions(rank):', rank_4_tensor.ndim)
print('Shape of tensor:', rank_4_tensor.shape)
print('Elements along the 0 axis:', rank_4_tensor.shape[0])
print('Elements along the last axis:', rank_4_tensor.shape[-1])
print('Total number of elements in our tensor:', tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions(rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: 120


# Indexing tensors

In [29]:
# Get the first two elemets of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [30]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[0, 0, 0, :]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>

In [31]:
rank_2_tensor = tf.constant([[10, 7],
                            [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [32]:
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [33]:
# Adding extra dimensions in our tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [34]:
tf.expand_dims(rank_2_tensor, axis = -1) #expanding dimension at the end

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [35]:
tf.expand_dims(rank_2_tensor, axis = 0) # expanding the first dimension

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]], dtype=int32)>

# Manipulating tensors

In [36]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7],
                      [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [37]:
tensor # since no assignment the values will not be changed

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [38]:
# Multiplication
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [39]:
# Subtraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [40]:
# we can use the tensorflow built-in function
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

# Matrix Multiplication

In [41]:
tf.matmul(tensor, tensor)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [42]:
tensor_A = tf.constant([[1, 2, 5],
                        [7, 2, 1],
                        [3, 3, 3]])

tensor_B = tf.constant([[3, 5],
                        [6, 7],
                        [1, 8]])

In [43]:
print(f'Tensor Dtype:\nTensor A: {tensor_A.dtype}\nTensor B: {tensor_B.dtype}\n')
print(f'Tensor Shape:\nTensor A: {tensor_A.shape}\nTensor B: {tensor_B.shape}\n')
print(f'Tensor Dimensions:\nTensor A: {tensor_A.ndim}\nTensor B: {tensor_B.ndim}\n')
print(f'Number of Tensor Elements:\nTensor A: {tf.size(tensor_A).numpy()}\nTensor B: {tf.size(tensor_B).numpy()}\n')

Tensor Dtype:
Tensor A: <dtype: 'int32'>
Tensor B: <dtype: 'int32'>

Tensor Shape:
Tensor A: (3, 3)
Tensor B: (3, 2)

Tensor Dimensions:
Tensor A: 2
Tensor B: 2

Number of Tensor Elements:
Tensor A: 9
Tensor B: 6



In [44]:
tensor_C = tf.matmul(tensor_A, tensor_B)
tensor_C

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [45]:
# can be also done with the '@' operator
tensor_A @ tensor_B

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

# TensorFlow and NumPy
* Difference is that the tensors can be run on GPU and TPU for faster processing

In [46]:
 # create a tensor directly from a Numpy array
 J = tf.constant(np.array([3., 7., 10.]))
 J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [47]:
# Convert our tensor to NumPy
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [48]:
# Can be also done like
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [49]:
# indexing with the resulting np array
J.numpy()[:2]

array([3., 7.])

In [50]:
# default types of each are slightly different
numpy_J = tf.constant(np.array([2., 8., 9., 10.]))
tensor_J = tf.constant([2., 8., 9., 10.])

numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

In [51]:
# Changing the shape of a tensor
tf.reshape(tensor_J, shape = (2, 2))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 2.,  8.],
       [ 9., 10.]], dtype=float32)>

In [52]:
# Transposing a tensor
tf.transpose(tf.reshape(tensor_J, shape = (2, 2)))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 2.,  9.],
       [ 8., 10.]], dtype=float32)>

# Changing the datatype of a tensor

In [53]:
# Create a tensor with the default datatype
B = tf.constant([1.7, 7.4])
C = tf.constant([1, 2])

B.dtype, C.dtype

(tf.float32, tf.int32)

In [54]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype = tf.float16)
E = tf.cast(C, dtype = tf.int16)

D.dtype, E.dtype

(tf.float16, tf.int16)

# Aggregrating tensors

In [55]:
# Get the absolute values
D = tf.constant([-28, -9])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([-28,  -9], dtype=int32)>

In [56]:
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([28,  9], dtype=int32)>

In [57]:
tf.random.set_seed(28)
tensor_A = tf.constant(np.random.randint(0, 100, size = 50))
tensor_A

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([13, 20,  7, 82, 14, 24, 19, 53,  7, 18,  7, 69, 43, 67, 61, 12, 56,
       43, 33, 53, 64, 59, 60, 80, 88, 32, 10,  0, 34, 59, 48, 24, 99,  5,
       69, 95, 28, 69, 75, 95, 86, 26, 72,  8, 21, 99, 40, 22, 76, 32])>

In [58]:
tf.reduce_min(tensor_A).numpy()

0

In [59]:
tf.reduce_max(tensor_A).numpy()

99

In [60]:
tf.reduce_mean(tensor_A).numpy()

45

In [61]:
tf.reduce_sum(tensor_A).numpy()

2276

In [69]:
tf.math.reduce_variance(tf.cast(tensor_A, dtype = tf.float64)).numpy()

843.2896000000001

In [66]:
tf.math.reduce_std(tf.cast(tensor_A, dtype = tf.float64)).numpy()

29.039449030585963

# Find Positional Maximum and Minimum

In [70]:
# Create a new tensor
tf.random.set_seed(28)
F = tf.random.uniform(shape = [50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.69515383, 0.9821805 , 0.20930278, 0.8134333 , 0.84545434,
       0.5072768 , 0.05098724, 0.251683  , 0.5644311 , 0.6561465 ,
       0.62229335, 0.01172853, 0.23984945, 0.6120213 , 0.00417602,
       0.867273  , 0.5918205 , 0.61515534, 0.51780415, 0.7553426 ,
       0.7759434 , 0.5330801 , 0.33834505, 0.8770453 , 0.6579268 ,
       0.13412297, 0.6838063 , 0.4704231 , 0.13656521, 0.11773098,
       0.61405945, 0.09456134, 0.64718986, 0.55908453, 0.90924907,
       0.6117854 , 0.23724854, 0.11413443, 0.95501363, 0.8245859 ,
       0.10976648, 0.89525795, 0.8205099 , 0.26334405, 0.94022703,
       0.4041524 , 0.23054874, 0.8307818 , 0.84843194, 0.05003941],
      dtype=float32)>

In [72]:
# Find the Positional maximum index
tf.argmax(F).numpy()

1

In [74]:
# Finding the value using the index
F[tf.argmax(F)].numpy()

0.9821805

In [75]:
# FInd the Positional minimum index
tf.argmin(F).numpy()

14

In [76]:
# Finding the value at the index
F[tf.argmin(F)].numpy()

0.0041760206

# Squeezing a tensor (removing all the single dimensions)

In [77]:
# Creating a tensor
tf.random.set_seed(28)
G = tf.constant(tf.random.uniform(shape = [50]), shape = [1, 1, 1, 1, 50])
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.69515383, 0.9821805 , 0.20930278, 0.8134333 , 0.84545434,
           0.5072768 , 0.05098724, 0.251683  , 0.5644311 , 0.6561465 ,
           0.62229335, 0.01172853, 0.23984945, 0.6120213 , 0.00417602,
           0.867273  , 0.5918205 , 0.61515534, 0.51780415, 0.7553426 ,
           0.7759434 , 0.5330801 , 0.33834505, 0.8770453 , 0.6579268 ,
           0.13412297, 0.6838063 , 0.4704231 , 0.13656521, 0.11773098,
           0.61405945, 0.09456134, 0.64718986, 0.55908453, 0.90924907,
           0.6117854 , 0.23724854, 0.11413443, 0.95501363, 0.8245859 ,
           0.10976648, 0.89525795, 0.8205099 , 0.26334405, 0.94022703,
           0.4041524 , 0.23054874, 0.8307818 , 0.84843194, 0.05003941]]]]],
      dtype=float32)>

In [79]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.69515383, 0.9821805 , 0.20930278, 0.8134333 , 0.84545434,
        0.5072768 , 0.05098724, 0.251683  , 0.5644311 , 0.6561465 ,
        0.62229335, 0.01172853, 0.23984945, 0.6120213 , 0.00417602,
        0.867273  , 0.5918205 , 0.61515534, 0.51780415, 0.7553426 ,
        0.7759434 , 0.5330801 , 0.33834505, 0.8770453 , 0.6579268 ,
        0.13412297, 0.6838063 , 0.4704231 , 0.13656521, 0.11773098,
        0.61405945, 0.09456134, 0.64718986, 0.55908453, 0.90924907,
        0.6117854 , 0.23724854, 0.11413443, 0.95501363, 0.8245859 ,
        0.10976648, 0.89525795, 0.8205099 , 0.26334405, 0.94022703,
        0.4041524 , 0.23054874, 0.8307818 , 0.84843194, 0.05003941],
       dtype=float32)>,
 TensorShape([50]))

# One-Hot Encoding

In [81]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue, purple

# One hot encode our list of indices
tf.one_hot(some_list, depth = 4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [82]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth = 4, on_value = 'deep learning is fun', off_value = 'but its tough')

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'deep learning is fun', b'but its tough', b'but its tough',
        b'but its tough'],
       [b'but its tough', b'deep learning is fun', b'but its tough',
        b'but its tough'],
       [b'but its tough', b'but its tough', b'deep learning is fun',
        b'but its tough'],
       [b'but its tough', b'but its tough', b'but its tough',
        b'deep learning is fun']], dtype=object)>

# Squaring, log, square root

In [83]:
# Create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [84]:
# Square of it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [85]:
# Square root of it
tf.sqrt(tf.cast(H, dtype = tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [87]:
# Finding the log
tf.math.log(tf.cast(H, dtype = tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

# Finding access to GPUs

In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
!nvidia-smi

Tue Mar 12 09:25:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--